<a href="https://colab.research.google.com/github/BadrishKhanna/NSAC_Terra/blob/main/Terra%20Data%20Compiller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

This jupyter notebook will comprise all the codes and details which are utilized to create our **Terra Data Compiler**

# **Checking the data**

In [ ]:
import io, requests
from datetime import datetime, timedelta
import numpy as np
from scipy.ndimage import gaussian_filter
from PIL import Image
import imageio.v2 as imageio

# -----------------------------
# CONFIG
# -----------------------------
WIDTH, HEIGHT = 800, 400
THERMAL_LAYER = "MODIS_Terra_Brightness_Temp_Band31_Day"
TRUE_LAYER    = "MODIS_Terra_CorrectedReflectance_TrueColor"

def placeholder_image(size=(WIDTH,HEIGHT)):
    return Image.new("RGB", size, (50,50,50))  # dark gray fallback

def fetch_global_image(date_str, layer):
    """Fetch MODIS global image for a given date and layer."""
    wms_url = (
        "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?"
        f"service=WMS&version=1.1.1&request=GetMap&layers={layer}&"
        f"styles=&format=image/png&transparent=FALSE&height={HEIGHT}&width={WIDTH}&"
        f"srs=EPSG:4326&bbox=-180,-90,180,90&time={date_str}"
    )
    try:
        r = requests.get(wms_url, timeout=20)
        if r.status_code == 200:
            return Image.open(io.BytesIO(r.content)).convert("RGB")
    except:
        pass
    return placeholder_image()

def world_to_pixels(lon, lat, size=(WIDTH,HEIGHT)):
    """Convert lon/lat to pixel coords on global map."""
    W,H = size
    x = int((lon+180)/360*W)
    y = int((90-lat)/180*H)
    return x,y

def render_hotspots(bg_img, day_idx, center):
    """Overlay growing red spots on thermal image."""
    W,H = bg_img.size
    heat = np.zeros((H,W),dtype=float)
    np.random.seed(42+day_idx)
    lon,lat = center
    num = 50 + day_idx*10  # number of hotspots grows with time
    for _ in range(num):
        jitter_lon = lon + np.random.randn()*2
        jitter_lat = lat + np.random.randn()*2
        x,y = world_to_pixels(jitter_lon,jitter_lat,(W,H))
        if 0<=x<W and 0<=y<H:
            heat[y,x]+=5
    heat = gaussian_filter(heat,sigma=3)
    overlay = Image.new("RGBA",(W,H),(0,0,0,0))
    ov = overlay.load()
    hmax = heat.max() if heat.max()>0 else 1
    for j in range(H):
        for i in range(W):
            v = heat[j,i]/hmax
            if v>0.1:
                ov[i,j]=(255,0,0,int(120+v*120))  # bright red
    return Image.alpha_composite(bg_img.convert("RGBA"),overlay).convert("RGB")

# -----------------------------
# FIRE EVENTS
# -----------------------------
FIRE_GROWTH_EVENTS = {
    "Amazon2019": {
        "start": "2019-08-01", "end": "2019-08-20",
        "center": (-60, -10)},
    "BlackSaturday2009": {
        "start": "2009-02-01", "end": "2009-02-12",
        "center": (146.5, -37.5)},
    "AugustComplex2020": {
        "start": "2020-08-15", "end": "2020-09-05",
        "center": (-122.5, 40.0)},
    "Texas2020": {
        "start": "2020-08-01", "end": "2020-08-15",
        "center": (-99, 31)},
}

# -----------------------------
# GENERATE MP4s
# -----------------------------
for ev,info in FIRE_GROWTH_EVENTS.items():
    sd=datetime.strptime(info["start"],"%Y-%m-%d")
    ed=datetime.strptime(info["end"],"%Y-%m-%d")

    thermal_frames=[]
    true_frames=[]
    day=0
    d=sd
    while d<=ed:
        ds=d.strftime("%Y-%m-%d")
        # Thermal + hotspots
        bg_th=fetch_global_image(ds,THERMAL_LAYER).resize((WIDTH,HEIGHT))
        frame_th=render_hotspots(bg_th,day,info["center"])
        thermal_frames.append(np.array(frame_th))
        # TrueColor
        bg_true=fetch_global_image(ds,TRUE_LAYER).resize((WIDTH,HEIGHT))
        true_frames.append(np.array(bg_true))
        d+=timedelta(days=1); day+=1

    # Save both animations
    out_file_th=f"{ev}_thermal.mp4"
    out_file_true=f"{ev}_true.mp4"
    imageio.mimsave(out_file_th,thermal_frames,fps=3,quality=8)
    imageio.mimsave(out_file_true,true_frames,fps=3,quality=8)
    print(f"✅ Saved {out_file_th} and {out_file_true}")

✅ Saved Amazon2019_thermal.mp4 and Amazon2019_true.mp4
✅ Saved BlackSaturday2009_thermal.mp4 and BlackSaturday2009_true.mp4
✅ Saved AugustComplex2020_thermal.mp4 and AugustComplex2020_true.mp4
✅ Saved Texas2020_thermal.mp4 and Texas2020_true.mp4


In [ ]:
!pip install -q pillow imageio imageio[ffmpeg] requests
import os, io, re, requests, imageio
from datetime import datetime, timedelta
from PIL import Image, ImageDraw

# ---------------- NASA GIBS fetcher ----------------
def fetch_gibs_wms(date_str, layer="MODIS_Terra_CorrectedReflectance_TrueColor",
                   width=1000, height=500, bbox="-180,-90,180,90"):
    """Fetch MODIS Terra TrueColor image from NASA GIBS for the given date."""
    url = (
        "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?"
        f"service=WMS&version=1.1.1&request=GetMap&layers={layer}&"
        f"styles=&format=image/png&transparent=FALSE&height={height}&width={width}&"
        f"srs=EPSG:4326&bbox={bbox}&time={date_str}"
    )
    try:
        r = requests.get(url, timeout=25)
        if r.status_code == 200:
            return Image.open(io.BytesIO(r.content)).convert("RGB")
    except Exception as e:
        print("⚠️ Error fetching", date_str, ":", e)
    img = Image.new("RGB", (width, height), (200, 200, 200))
    ImageDraw.Draw(img).text((20, 20), f"No data {date_str}", fill=(0, 0, 0))
    return img

# ---------------- Event bounding boxes ----------------
CYCLONE_EVENTS = {
    "2019-10-12": {"name":"Typhoon Hagibis, Japan – 2019","bbox":[137,32,142,38]},
    "2019-03-14": {"name":"Cyclone Idai, Mozambique/Zimbabwe – 2019","bbox":[34,-22,38,-17]},
    "2008-05-02": {"name":"Cyclone Nargis, Myanmar – 2008","bbox":[93,13,98,18]},
    "2013-11-08": {"name":"Typhoon Haiyan, Philippines – 2013","bbox":[122,9,126,13]},
    "2019-05-03": {"name":"Cyclone Fani, India – 2019","bbox":[83,17,87,21]},
    "2020-05-20": {"name":"Cyclone Amphan, India/Bangladesh – 2020","bbox":[86,18,91,23]},
    "2022-09-25": {"name":"Typhoon Noru, Philippines/Vietnam – 2022","bbox":[122,11,128,17]},
    "2021-05-17": {"name":"Cyclone Tauktae, India – 2021","bbox":[67,12,72,19]},
    "2018-09-14": {"name":"Typhoon Mangkhut, Philippines/China – 2018","bbox":[118,12,123,19]},
    "2013-10-12": {"name":"Cyclone Phailin, India – 2013","bbox":[84,17,88,21]},
}

# ---------------- Draw bounding box ----------------
def draw_bbox(img, bbox, label=None, color=(255, 0, 0), width=3):
    minLon, minLat, maxLon, maxLat = bbox
    w, h = img.size
    x1 = int((minLon + 180) / 360 * w)
    x2 = int((maxLon + 180) / 360 * w)
    y1 = int((90 - maxLat) / 180 * h)
    y2 = int((90 - minLat) / 180 * h)
    draw = ImageDraw.Draw(img)
    for i in range(width):
        draw.rectangle([x1 - i, y1 - i, x2 + i, y2 + i], outline=color)
    if label:
        draw.rectangle([x1, y1 - 20, x1 + 220, y1], fill=(0, 0, 0))
        draw.text((x1 + 5, y1 - 18), label, fill=(255, 255, 255))
    return img

# ---------------- Create and save animation (MP4) ----------------
def make_cyclone_animation(date_str, event, days_before=2, days_after=2, out_dir="cyclone_animations"):
    os.makedirs(out_dir, exist_ok=True)
    center_date = datetime.strptime(date_str, "%Y-%m-%d")
    frames = []
    for offset in range(-days_before, days_after + 1):
        d = center_date + timedelta(days=offset)
        ds = d.strftime("%Y-%m-%d")
        img = fetch_gibs_wms(ds)
        img = draw_bbox(img, event["bbox"], label=f"{event['name']} ({ds})")
        frames.append(img)

    safe_name = re.sub(r"[^A-Za-z0-9_-]", "_", event["name"])
    mp4_path = os.path.join(out_dir, f"{safe_name}_{date_str}.mp4")

    imageio.mimsave(mp4_path, frames, fps=1)
    print(f"🎥 Saved MP4: {mp4_path}")

# ---------------- Run all animations ----------------
for date, ev in CYCLONE_EVENTS.items():
    make_cyclone_animation(date, ev)

# **Final**

In [ ]:
# ------------------------------
# Install dependencies
# ------------------------------
!pip install -q flask flask-cors pyngrok pillow requests imageio

# ------------------------------
# Imports
# ------------------------------
from flask import Flask, request, send_file, jsonify, Response
from flask_cors import CORS
from pyngrok import ngrok
import requests, io, logging, os, zipfile
from PIL import Image, ImageDraw
from datetime import datetime, timedelta
import random
import numpy as np

logging.basicConfig(level=logging.INFO)

# ------------------------------
# Flask App
# ------------------------------
app = Flask(__name__)
CORS(app)

# ------------------------------
# CONFIG
# ------------------------------
NGROK_AUTH_TOKEN = "32u5TjkQusOqJpzUEzFUho2V1fD_83nFrc4wTVizqvjZa9U5L"
TRUE_LAYER = "MODIS_Terra_CorrectedReflectance_TrueColor"
THERMAL_LAYER = "MODIS_Terra_Brightness_Temp_Band31_Day"
WIDTH, HEIGHT = 1000, 500
EVENT_WIDTH, EVENT_HEIGHT = 480, 360

MOPITT_START_YEAR = 2000
MOPITT_START_MONTH = 3
mopitt_images = []

# ------------------------------
# Event dictionaries
# ------------------------------
FIRE_EVENTS = {
    "2023-08-08": {
        "name": "Maui Wildfires, USA – 2023",
        "bbox": [-157.1, 20.7, -156.3, 21.1],
        "description": "A catastrophic wildfire that swept through Lahaina on the island of Maui, Hawaii, destroying much of the town and causing significant loss of life."
    },
    "2020-09-10": {
        "name": "August Complex Fire, California, USA – 2020",
        "bbox": [-124, 39, -122, 41],
        "description": "The largest wildfire in California's history, burning over a million acres across Northern California."
    },
    "2024-02-28": {
        "name": "Texas Wildfires, USA – 2024",
        "bbox": [-103.5, 34, -100, 36],
        "description": "A major wildfire outbreak in the Texas Panhandle, including the Smokehouse Creek Fire, which became the largest in state history."
    },
    "2018-11-08": {
        "name": "Camp Fire, California, USA – 2018",
        "bbox": [-122, 39.5, -121, 40],
        "description": "The deadliest and most destructive wildfire in California’s history, destroying the town of Paradise."
    },
    "2019-08-15": {
        "name": "Amazon Rainforest Fires, Brazil – 2019",
        "bbox": [-75, -15, -45, 5],
        "description": "A massive increase in deforestation fires across the Amazon rainforest, drawing global attention to environmental damage."
    },
    "2019-12-15": {
        "name": "Australia Bushfires (Black Summer) – 2019–2020",
        "bbox": [140, -42, 154, -25],
        "description": "One of Australia's worst bushfire seasons, destroying millions of hectares and killing or displacing nearly 3 billion animals."
    },
    "2017-10-08": {
        "name": "Tubbs Fire, California, USA – 2017",
        "bbox": [-123.1, 38.3, -122.4, 38.7],
        "description": "A fast-moving fire in Northern California that devastated Santa Rosa and became one of the most destructive fires in the state."
    },
    "2018-11-09": {
        "name": "Woolsey Fire, California, USA – 2018",
        "bbox": [-119, 34, -118.5, 34.3],
        "description": "A destructive wildfire that swept through Malibu and Los Angeles County, fueled by Santa Ana winds."
    },
    "2015-03-15": {
        "name": "Chile Wildfires – 2015",
        "bbox": [-72, -36, -70, -33],
        "description": "Large wildfires that burned through central Chile during a period of drought, affecting forests and agricultural regions."
    },
    "2009-02-07": {
        "name": "Black Saturday Bushfires, Australia – 2009",
        "bbox": [144, -38.5, 147, -36],
        "description": "A series of devastating bushfires in Victoria, Australia, killing 173 people and destroying thousands of homes."
    }
}

CYCLONE_EVENTS = {
    "2019-10-12": {
        "name": "Typhoon Hagibis, Japan – 2019",
        "bbox": [137, 32, 142, 38],
        "description": "One of the strongest typhoons to hit Japan in decades, causing widespread flooding and damage."
    },
    "2019-03-14": {
        "name": "Cyclone Idai, Mozambique/Zimbabwe – 2019",
        "bbox": [34, -22, 38, -17],
        "description": "A devastating cyclone that brought catastrophic flooding to southeastern Africa."
    },
    "2008-05-02": {
        "name": "Cyclone Nargis, Myanmar – 2008",
        "bbox": [93, 13, 98, 18],
        "description": "A deadly cyclone that hit Myanmar’s Irrawaddy Delta, killing over 130,000 people."
    },
    "2013-11-08": {
        "name": "Typhoon Haiyan, Philippines – 2013",
        "bbox": [122, 9, 126, 13],
        "description": "One of the strongest tropical cyclones ever recorded, devastating the central Philippines."
    },
    "2019-05-03": {
        "name": "Cyclone Fani, India – 2019",
        "bbox": [83, 17, 87, 21],
        "description": "A strong tropical cyclone that caused major destruction along India’s east coast."
    },
    "2020-05-20": {
        "name": "Cyclone Amphan, India/Bangladesh – 2020",
        "bbox": [86, 18, 91, 23],
        "description": "A super cyclone that struck eastern India and Bangladesh, causing massive damage."
    },
    "2022-09-25": {
        "name": "Typhoon Noru, Philippines/Vietnam – 2022",
        "bbox": [122, 11, 128, 17],
        "description": "A powerful typhoon that impacted the Philippines and Vietnam with heavy rainfall and winds."
    },
    "2021-05-17": {
        "name": "Cyclone Tauktae, India – 2021",
        "bbox": [67, 12, 72, 19],
        "description": "An intense Arabian Sea cyclone that battered India’s western coast."
    },
    "2018-09-14": {
        "name": "Typhoon Mangkhut, Philippines/China – 2018",
        "bbox": [118, 12, 123, 19],
        "description": "A violent typhoon that caused severe flooding across the Philippines and southern China."
    },
    "2013-10-12": {
        "name": "Cyclone Phailin, India – 2013",
        "bbox": [84, 17, 88, 21],
        "description": "A very severe cyclone that struck Odisha and Andhra Pradesh, prompting mass evacuations."
    }
}

TSUNAMI_EVENTS = {
    "2004-12-26": {
        "name": "Indian Ocean Tsunami, Southeast Asia – 2004",
        "bbox": [90, -5, 105, 15],
        "description": "One of the deadliest natural disasters in history, triggered by a massive earthquake near Sumatra."
    },
    "2011-03-11": {
        "name": "Tōhoku Tsunami, Japan – 2011",
        "bbox": [140, 36, 145, 40],
        "description": "A massive tsunami following a magnitude 9.0 earthquake off Japan’s east coast, causing widespread devastation and the Fukushima disaster."
    },
    "2009-09-29": {
        "name": "Samoa Tsunami, Pacific – 2009",
        "bbox": [-174, -16, -170, -13],
        "description": "A powerful undersea earthquake generated a tsunami that struck Samoa, American Samoa, and Tonga."
    },
    "2018-12-22": {
        "name": "Sunda Strait Tsunami, Indonesia – 2018",
        "bbox": [104, -7, 107, -5],
        "description": "A tsunami caused by the partial collapse of Anak Krakatau volcano, hitting coastal Indonesia without warning."
    },
    "2006-07-17": {
        "name": "Java Tsunami, Indonesia – 2006",
        "bbox": [106, -9, 110, -6],
        "description": "A tsunami generated by a magnitude 7.7 earthquake struck southern Java, causing heavy casualties."
    },
    "2005-03-28": {
        "name": "Nias–Simeulue Tsunami, Indonesia – 2005",
        "bbox": [95, 0, 99, 4],
        "description": "A major tsunami caused by an 8.6-magnitude earthquake off northern Sumatra."
    },
    "2010-02-27": {
        "name": "Chile Tsunami – 2010",
        "bbox": [-75, -40, -70, -30],
        "description": "A massive tsunami caused by an 8.8-magnitude earthquake off Chile’s coast, reaching as far as Japan."
    },
    "2007-04-02": {
        "name": "Solomon Islands Tsunami – 2007",
        "bbox": [155, -9, 159, -6],
        "description": "A strong undersea earthquake triggered a tsunami that struck the Solomon Islands and nearby regions."
    },
    "2014-04-01": {
        "name": "Papua New Guinea Tsunami – 2014",
        "bbox": [146, -6, 151, -3],
        "description": "A tsunami caused by a 7.7-magnitude earthquake near Papua New Guinea’s northern coast."
    },
    "2021-07-29": {
        "name": "Alaska Tsunami, USA – 2021",
        "bbox": [-157, 55, -149, 60],
        "description": "A tsunami warning was issued after a magnitude 8.2 earthquake struck off Alaska’s coast; waves were observed but damage was limited."
    }
}
# ------------------------------
# Helpers
# ------------------------------
def placeholder_image(size=(WIDTH, HEIGHT), text=None):
    img = Image.new("RGB", size, (240,240,240))
    if text:
        draw = ImageDraw.Draw(img)
        draw.text((10,10), text, fill=(60,60,60))
    return img

def fetch_gibs_wms(date_str, layer, width=WIDTH, height=HEIGHT, bbox="-180,-90,180,90"):
    url = ("https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?"
           f"service=WMS&version=1.1.1&request=GetMap&layers={layer}&"
           f"styles=&format=image/png&transparent=FALSE&height={height}&width={width}&"
           f"srs=EPSG:4326&bbox={bbox}&time={date_str}")
    try:
        r = requests.get(url, timeout=20)
        if r.status_code == 200:
            return Image.open(io.BytesIO(r.content)).convert("RGB")
    except:
        pass
    return placeholder_image((width, height), text="No data")

# ------------------------------
# MOPITT handling
# ------------------------------
import gdown
import os
import zipfile
import logging

# Path where the ZIP will be saved
MOPITT_ZIP_PATH = "mopitt_images.zip"

# Google Drive file ID
file_id = "1CxcS7VlcCL2x9KgzKp2SMOpnPvxhm2iR"

# File for MOPITT Data
url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Download the file if it doesn't exist
if not os.path.exists(MOPITT_ZIP_PATH):
    logging.info("Downloading MOPITT ZIP from Google Drive...")
    gdown.download(url, MOPITT_ZIP_PATH, quiet=False)
else:
    logging.info("MOPITT ZIP already exists, skipping download.")


def load_mopitt_zip():
    global mopitt_images
    mopitt_images = []
    extract_dir = "mopitt_frames"
    os.makedirs(extract_dir, exist_ok=True)
    with zipfile.ZipFile(MOPITT_ZIP_PATH, 'r') as z:
        jpg_files = [f for f in z.namelist() if f.lower().endswith('.jpg')]
        jpg_files.sort()
        for f in jpg_files:
            z.extract(f, extract_dir)
            mopitt_images.append(os.path.join(extract_dir, f))
    logging.info("Loaded %d MOPITT images", len(mopitt_images))

load_mopitt_zip()

def mopitt_index_for_year_month(year, month):
    return (year - MOPITT_START_YEAR) * 12 + (month - MOPITT_START_MONTH)

# ------------------------------
# HTML Frontend
# ------------------------------
@app.route("/")
def index():
    try: public_url = ngrok.get_tunnels()[0].public_url
    except: public_url = "http://localhost:5000"
    html = f"""<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>MODIS + Thermal + MOPITT Viewer</title>
<style>
body{{font-family:Segoe UI,Helvetica,Arial,sans-serif;background:#f0f2f5;margin:0;color:#1f2933}}
header{{background:linear-gradient(135deg,#273c75,#192a56);color:#fff;padding:18px;text-align:center}}
.container{{max-width:1200px;margin:24px auto;padding:20px;background:#fff;border-radius:10px;box-shadow:0 8px 24px rgba(0,0,0,0.08)}}
.controls{{display:flex;gap:12px;flex-wrap:wrap;align-items:center}}
.controls label{{font-weight:600}}
input,select,button{{padding:8px 10px;border-radius:6px;border:1px solid #d1d5db;font-size:14px}}
button{{background:#0b63d6;color:#fff;border:none;cursor:pointer}}
.tabs{{margin-top:16px}}
.tab-buttons button{{margin-right:8px;padding:8px 14px;background:#e6eefc;border-radius:6px;border:none;cursor:pointer}}
.tab-buttons button.active{{background:#273c75;color:#fff}}
.view{{display:none;margin-top:18px}}
.view.active{{display:block}}
.layout{{display:flex;gap:20px}}
.main{{flex:7;}}
.side{{flex:3;max-width:380px;}}
img{{width:100%;border-radius:8px;box-shadow:0 6px 18px rgba(0,0,0,0.08)}}
.note{{color:#555;margin-top:8px;font-size:13px}}
.footer{{margin-top:14px;color:#666;font-size:13px}}
</style>
</head>
<body>
<header><h1>Terra Data Compiler</h1></header>
<div class="container">
  <div class="controls">
    <label>Single date:</label><input type="date" id="single-date">
    <label>Start:</label><input type="date" id="start-date">
    <label>End:</label><input type="date" id="end-date">
    <label>MOPITT month:</label><select id="mopitt-month"></select>
    <button onclick="showCurrent()">Show</button>
  </div>
  <div class="tabs">
    <div class="tab-buttons">
      <button class="active" onclick="switchTab('truecolor',event)">True Color</button>
      <button onclick="switchTab('thermal',event)">Thermal / Fire</button>
      <button onclick="switchTab('mopitt',event)">MOPITT (monthly)</button>
      <button onclick="switchTab('tsunami',event)">Tsunami</button>
    </div>
    <div id="truecolor" class="view active">
      <h3>True Color (MODIS)</h3>
      <div class="layout">
        <div class="main" id="true-main"><div class="note">No image loaded</div></div>
        <div class="side" id="true-side"><div class="note">Event preview</div></div>
      </div>
    </div>
    <div id="thermal" class="view">
      <h3>Thermal (Band 31) & Fire Hotspots</h3>
      <div class="layout">
        <div class="main" id="therm-main"><div class="note">No image loaded</div></div>
        <div class="side" id="therm-side"><div class="note">Fire preview</div></div>
      </div>
    </div>
    <div id="mopitt" class="view">
      <h3>MOPITT CO (monthly JPGs from ZIP)</h3>
      <div class="layout">
        <div class="main" id="mopitt-main"><div class="note">No data loaded</div></div>
        <div class="side" id="mopitt-side"><div class="note">MOPITT info</div></div>
      </div>
    </div>
    <div id="tsunami" class="view">
      <h3>Tsunami (preview + zoom)</h3>
      <div class="layout">
        <div class="main" id="tsunami-main"><div class="note">No image loaded</div></div>
        <div class="side" id="tsunami-side"><div class="note">Tsunami zoom</div></div>
      </div>
    </div>
  </div>
  <div class="footer">Backend base URL: <strong>{public_url}</strong></div>
</div>

<script>
let current='truecolor';
function switchTab(name,e){{
  current=name;
  document.querySelectorAll('.tab-buttons button').forEach(b=>b.classList.remove('active'));
  e.target.classList.add('active');
  document.querySelectorAll('.view').forEach(v=>v.classList.remove('active'));
  document.getElementById(name).classList.add('active');
}}

function populateMopittDropdown(){{
  const dropdown=document.getElementById('mopitt-month');
  dropdown.innerHTML='';
  const startYear=2000;
  const startMonth=3;
  const totalMonths={len(mopitt_images)};
  for(let i=0;i<totalMonths;i++){{
    const total=(startMonth-1)+i;
    const year=startYear + Math.floor(total/12);
    const month=(total%12)+1;
    const opt=document.createElement('option');
    opt.value=i;
    opt.textContent=`${{year}}-${{month.toString().padStart(2,'0')}}`;
    dropdown.appendChild(opt);
  }}
}}
document.addEventListener('DOMContentLoaded', populateMopittDropdown);

function showCurrent(){{
  if(current==='truecolor') showTrue();
  else if(current==='thermal') showThermal();
  else if(current==='mopitt') showMopitt();
  else if(current==='tsunami') showTsunami();
}}

function showTrue(){{
  const d=document.getElementById('single-date').value;
  const s=document.getElementById('start-date').value;
  const e=document.getElementById('end-date').value;
  const main=document.getElementById('true-main');
  const side=document.getElementById('true-side');
  main.innerHTML=''; side.innerHTML='';
  if(s && e){{
    const img=document.createElement('img'); img.src='/true_anim?start='+s+'&end='+e;
    main.appendChild(img);
  }} else if(d){{
    const img=document.createElement('img'); img.src='/true_image?date='+d;
    main.appendChild(img);
    fetch('/true_event?date='+d).then(r=>r.json()).then(info=>{{
      if(info.found){{
        side.innerHTML='<h4>'+info.name+'</h4><p>'+info.description+'</p><img src="/true_event_image?date='+d+'">';
      }}
    }});
  }} else {{ main.innerHTML='<div class="note">Select a date or range.</div>'; }}
}}

function showThermal(){{
  const d=document.getElementById('single-date').value;
  const s=document.getElementById('start-date').value;
  const e=document.getElementById('end-date').value;
  const main=document.getElementById('therm-main');
  const side=document.getElementById('therm-side');
  main.innerHTML=''; side.innerHTML='';
  if(s && e){{
    const img=document.createElement('img'); img.src='/therm_anim?start='+s+'&end='+e;
    main.appendChild(img);
  }} else if(d){{
    const img=document.createElement('img'); img.src='/therm_image?date='+d;
    main.appendChild(img);
    fetch('/fire_event?date='+d).then(r=>r.json()).then(info=>{{
      if(info.found){{
        side.innerHTML='<h4>'+info.name+'</h4><p>'+info.description+'</p><img src="/fire_event_image?date='+d+'">';
      }}
    }});
  }} else {{ main.innerHTML='<div class="note">Select a date or range.</div>'; }}
}}

function showMopitt(){{
  const mv=document.getElementById('mopitt-month').value;
  const main=document.getElementById('mopitt-main');
  main.innerHTML='';
  if(mv!==''){{
    main.innerHTML='<img src="/mopitt_image?idx='+mv+'">';
  }} else {{ main.innerHTML='<div class="note">Select a month (MOPITT)</div>'; }}
}}

function showTsunami(){{
  const d=document.getElementById('single-date').value;
  const s=document.getElementById('start-date').value;
  const e=document.getElementById('end-date').value;
  const main=document.getElementById('tsunami-main');
  const side=document.getElementById('tsunami-side');
  main.innerHTML=''; side.innerHTML='';
  if(s && e){{
    const img=document.createElement('img'); img.src='/tsunami_anim?start='+s+'&end='+e;
    main.appendChild(img);
  }} else if(d){{
    const img=document.createElement('img'); img.src='/tsunami_image?date='+d;
    main.appendChild(img);
    fetch('/tsunami_event?date='+d).then(r=>r.json()).then(info=>{{
      if(info.found){{
        side.innerHTML='<h4>'+info.name+'</h4><p>'+info.description+'</p><img src="/tsunami_event_image?date='+d+'">';
      }}
    }});
  }} else {{ main.innerHTML='<div class="note">Select a date or range.</div>'; }}
}}
</script>
</body></html>"""
    return Response(html,mimetype="text/html")

# ------------------------------
# MODIS / True Color endpoints
# ------------------------------
@app.route("/true_image")
def true_image():
    date = request.args.get("date")
    if not date: return "Missing date",400
    img = fetch_gibs_wms(date, TRUE_LAYER)
    buf=io.BytesIO()
    img.save(buf,"PNG"); buf.seek(0)
    return send_file(buf,mimetype="image/png")

@app.route("/true_anim")
def true_anim():
    start = request.args.get("start"); end = request.args.get("end")
    if not start or not end: return "Missing start/end",400
    sd = datetime.strptime(start,"%Y-%m-%d")
    ed = datetime.strptime(end,"%Y-%m-%d")
    frames=[]
    d=sd
    while d<=ed:
        frames.append(fetch_gibs_wms(d.strftime("%Y-%m-%d"),TRUE_LAYER).convert("P",palette=Image.ADAPTIVE))
        d += timedelta(days=1)
    buf=io.BytesIO()
    frames[0].save(buf,format='GIF', save_all=True, append_images=frames[1:], duration=600, loop=0)
    buf.seek(0)
    return send_file(buf,mimetype="image/gif")

@app.route("/true_event")
def true_event():
    date=request.args.get("date")
    ev=CYCLONE_EVENTS.get(date)
    if not ev: return jsonify({"found":False})
    return jsonify({"found":True,"name":ev["name"],"description":ev["description"],"bbox":ev["bbox"]})

@app.route("/true_event_image")
def true_event_image():
    from PIL import ImageDraw, ImageEnhance, ImageFilter

    date = request.args.get("date")
    ev = CYCLONE_EVENTS.get(date)
    if not ev:
        return "No event", 404

    g = fetch_gibs_wms(date, TRUE_LAYER)
    minLon, minLat, maxLon, maxLat = ev["bbox"]
    w, h = g.size

    x1 = max(0, int((minLon + 180) / 360 * w))
    x2 = min(w, int((maxLon + 180) / 360 * w))
    y1 = max(0, int((90 - maxLat) / 180 * h))
    y2 = min(h, int((90 - minLat) / 180 * h))

    padx = int((x2 - x1) * 1.0)
    pady = int((y2 - y1) * 1.0)
    x1 = max(0, x1 - padx)
    x2 = min(w, x2 + padx)
    y1 = max(0, y1 - pady)
    y2 = min(h, y2 + pady)

    region = g.crop((x1, y1, x2, y2)).resize((EVENT_WIDTH, EVENT_HEIGHT), Image.LANCZOS)
    region = ImageEnhance.Contrast(region).enhance(1.25)

    overlay = Image.new("RGBA", region.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)
    cx, cy = EVENT_WIDTH // 2, EVENT_HEIGHT // 2

    for r in range(60, 180, 30):
        draw.ellipse((cx - r, cy - r, cx + r, cy + r), outline=(255, 255, 255, 90), width=3)
    overlay = overlay.filter(ImageFilter.GaussianBlur(6))

    final_img = Image.alpha_composite(region.convert("RGBA"), overlay)

    # Label overlay
    draw = ImageDraw.Draw(final_img)
    draw.rectangle([10, 10, 520, 65], fill=(0, 0, 0, 130))
    draw.text((20, 28), f"{ev['name']} ({date})", fill=(255, 255, 255, 255))

    buf = io.BytesIO()
    final_img.save(buf, "PNG")
    buf.seek(0)
    return send_file(buf, mimetype="image/png")

# ------------------------------
# Thermal / Fire endpoints
# ------------------------------
@app.route("/therm_image")
def therm_image():
    date=request.args.get("date")
    if not date: return "Missing date",400
    img=fetch_gibs_wms(date,THERMAL_LAYER)
    buf=io.BytesIO(); img.save(buf,"PNG"); buf.seek(0)
    return send_file(buf,mimetype="image/png")

@app.route("/therm_anim")
def therm_anim():
    start=request.args.get("start"); end=request.args.get("end")
    if not start or not end: return "Missing start/end",400
    sd=datetime.strptime(start,"%Y-%m-%d"); ed=datetime.strptime(end,"%Y-%m-%d")
    frames=[]; d=sd
    while d<=ed:
        frames.append(fetch_gibs_wms(d.strftime("%Y-%m-%d"),THERMAL_LAYER).convert("P",palette=Image.ADAPTIVE))
        d += timedelta(days=1)
    buf=io.BytesIO()
    frames[0].save(buf, format='GIF', save_all=True, append_images=frames[1:], duration=600, loop=0)
    buf.seek(0)
    return send_file(buf, mimetype="image/gif")

@app.route("/fire_event")
def fire_event():
    date = request.args.get("date")
    ev = FIRE_EVENTS.get(date)
    if not ev:
        return jsonify({"found": False})
    return jsonify({"found": True, "name": ev["name"], "description": ev["description"], "bbox": ev["bbox"]})

@app.route("/fire_event_image")
def fire_event_image():
    from PIL import ImageDraw, ImageEnhance, ImageFilter

    date = request.args.get("date")
    ev = FIRE_EVENTS.get(date)
    if not ev:
        return "No event", 404

    # Fetch thermal MODIS layer (shows heat signatures)
    g = fetch_gibs_wms(date, THERMAL_LAYER)

    minLon, minLat, maxLon, maxLat = ev["bbox"]
    w, h = g.size

    x1 = max(0, int((minLon + 180) / 360 * w))
    x2 = min(w, int((maxLon + 180) / 360 * w))
    y1 = max(0, int((90 - maxLat) / 180 * h))
    y2 = min(h, int((90 - minLat) / 180 * h))

    # Make view slightly wider so we can see surrounding areas
    padx = int((x2 - x1) * 1)
    pady = int((y2 - y1) * 1)
    x1 = max(0, x1 - padx)
    x2 = min(w, x2 + padx)
    y1 = max(0, y1 - pady)
    y2 = min(h, y2 + pady)

    # Event image ratio
    region = g.crop((x1, y1, x2, y2)).resize((EVENT_WIDTH, EVENT_HEIGHT), Image.LANCZOS)
    region = ImageEnhance.Contrast(region).enhance(1.4)
    region = ImageEnhance.Color(region).enhance(1.3)

    # Glowing “hotspot” overlay
    overlay = Image.new("RGBA", region.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)

    cx, cy = EVENT_WIDTH // 2, EVENT_HEIGHT // 2
    for _ in range(60):
        # Spread fires roughly around center
        rx = np.random.normal(cx, EVENT_WIDTH / 5)
        ry = np.random.normal(cy, EVENT_HEIGHT / 5)
        rsize = np.random.randint(4, 12)
        color = (255, np.random.randint(60, 180), 0, np.random.randint(120, 220))
        draw.ellipse([rx - rsize, ry - rsize, rx + rsize, ry + rsize], fill=color)

    # Add blur for glowing effect
    overlay = overlay.filter(ImageFilter.GaussianBlur(5))
    blended = Image.alpha_composite(region.convert("RGBA"), overlay)

    #Subtle smoke haze (gray transparent overlay)
    smoke = Image.new("RGBA", region.size, (80, 80, 80, 50))
    blended = Image.alpha_composite(blended, smoke)

    # Add title bar
    draw = ImageDraw.Draw(blended)
    draw.rectangle([10, 10, 520, 65], fill=(0, 0, 0, 130))
    draw.text((20, 28), f"{ev['name']} ({date})", fill=(255, 255, 255, 255))

    buf = io.BytesIO()
    blended.save(buf, "PNG")
    buf.seek(0)
    return send_file(buf, mimetype="image/png")

# ------------------------------
# MOPITT endpoints
# ------------------------------
@app.route("/mopitt_image")
def mopitt_image():
    idx = request.args.get("idx")
    try:
        idx = int(idx)
    except:
        return "Invalid index",400
    if idx<0 or idx>=len(mopitt_images):
        return "No image",404
    return send_file(mopitt_images[idx], mimetype="image/jpeg")

# ------------------------------
# Tsunami endpoints
# ------------------------------
@app.route("/tsunami_image")
def tsunami_image():
    date = request.args.get("date")
    if not date: return "Missing date",400
    img = fetch_gibs_wms(date, TRUE_LAYER)
    buf=io.BytesIO(); img.save(buf,"PNG"); buf.seek(0)
    return send_file(buf,mimetype="image/png")

@app.route("/tsunami_event")
def tsunami_event():
    date = request.args.get("date")
    ev = TSUNAMI_EVENTS.get(date)
    if not ev: return jsonify({"found":False})
    return jsonify({"found":True,"name":ev["name"],"description":ev["description"],"bbox":ev["bbox"]})

@app.route("/tsunami_event_image")
def tsunami_event_image():
    from PIL import ImageDraw, ImageEnhance, ImageFilter

    date = request.args.get("date")
    ev = TSUNAMI_EVENTS.get(date)
    if not ev:
        return "No event", 404

    g = fetch_gibs_wms(date, TRUE_LAYER)
    minLon, minLat, maxLon, maxLat = ev["bbox"]
    w, h = g.size

    #Coordinates
    x1 = max(0, int((minLon + 180) / 360 * w))
    x2 = min(w, int((maxLon + 180) / 360 * w))
    y1 = max(0, int((90 - maxLat) / 180 * h))
    y2 = min(h, int((90 - minLat) / 180 * h))

    #Event Image Ratio
    padx = int((x2 - x1) * 1.2)
    pady = int((y2 - y1) * 1.2)
    x1 = max(0, x1 - padx)
    x2 = min(w, x2 + padx)
    y1 = max(0, y1 - pady)
    y2 = min(h, y2 + pady)

    region = g.crop((x1, y1, x2, y2)).resize((EVENT_WIDTH, EVENT_HEIGHT), Image.LANCZOS)
    region = ImageEnhance.Color(region).enhance(1.2)

    overlay = Image.new("RGBA", region.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)
    cx, cy = EVENT_WIDTH // 2, EVENT_HEIGHT // 2
    for r in range(60, 240, 30):
        draw.ellipse((cx - r, cy - r, cx + r, cy + r), outline=(0, 100, 255, 80), width=5)
    overlay = overlay.filter(ImageFilter.GaussianBlur(10))

    final_img = Image.alpha_composite(region.convert("RGBA"), overlay)
    draw = ImageDraw.Draw(final_img)
    draw.rectangle([10, 10, 450, 60], fill=(0, 0, 0, 120))
    draw.text((20, 25), f"{ev['name']} ({date})", fill=(255, 255, 255, 255))

    buf = io.BytesIO()
    final_img.save(buf, "PNG")
    buf.seek(0)
    return send_file(buf, mimetype="image/png")

@app.route("/tsunami_anim")
def tsunami_anim():
    start = request.args.get("start"); end = request.args.get("end")
    if not start or not end: return "Missing start/end",400
    sd=datetime.strptime(start,"%Y-%m-%d"); ed=datetime.strptime(end,"%Y-%m-%d")
    frames=[]; d=sd
    while d<=ed:
        frames.append(fetch_gibs_wms(d.strftime("%Y-%m-%d"),TRUE_LAYER).convert("P",palette=Image.ADAPTIVE))
        d += timedelta(days=1)
    buf=io.BytesIO()
    frames[0].save(buf,format='GIF', save_all=True, append_images=frames[1:], duration=600, loop=0)
    buf.seek(0)
    return send_file(buf,mimetype="image/gif")

# ------------------------------
# Flask + ngrok initiation
# ------------------------------
def start_server():
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    ngrok.kill()
    public_url = ngrok.connect(5000)
    print("🌍 Public URL:", public_url)
    app.run(host="0.0.0.0", port=5000)

# ------------------------------
# Run server
# ------------------------------
if __name__=="__main__":
    start_server()